In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import os
import requests
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import ndimage
from skimage.color import rgb2hsv
import skimage
from PIL import Image
import random
import cv2
import scipy.misc
from skimage import data, io
import pandas as pd

from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2gray

from sklearn.model_selection import train_test_split

In [2]:
pp = pd.read_csv('../data/picasso.csv')
hm = pd.read_csv('../data/henri_dic.csv')
from skimage import data, io

In [3]:
def RMS_CONTRAST(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    oneD_pixels = np.concatenate(img, axis=0)

    brightness = 0
    ## average brightness
    for info in oneD_pixels:
        brightness += (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
    average_brightness = brightness / len(oneD_pixels)  ## calculate average
    average_brightness /= 255  ## normalizing to 0 and 1

    rms = 0
    for info in oneD_pixels:
        brightness = (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
        brightness /= 255  ## normalizing to 0 and 1
        rms += pow((brightness - average_brightness), 2);  ## calculate squared
    rms /= len(oneD_pixels)  ## calculate mean squared
    rms = pow(rms, 0.5)
    return rms

def average_entropy(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    gray_img = rgb2gray(img)
    return entropy(gray_img, disk(5)).mean()

def calc_stats_mat(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,1]
    return result

def calc_stats_pic(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,0]
    return result

In [4]:
# your code here
mat_dir = r'..\data\henri-matisse'
pic_dir = r'..\data\picasso'
db={}
for filename in os.listdir(mat_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(mat_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_mat(temp_picture)
        db[filename]=temp_result
    else:
        continue
for filename in os.listdir(pic_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(pic_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_pic(temp_picture)
        db[filename]=temp_result
    else:
        continue

In [5]:
dataframe= pd.DataFrame.from_dict(db, orient='index',
columns=['width', 'height', 'mean_hue', 'mean_saturation','mean_brightness','variance_brightness','sum_edge','matis'])
dataframe["Picturesource"] = dataframe.index
dataframe

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource
paintings-orient_107_thmb.jpg,150,119,0.245508,0.419425,0.699409,0.057506,0.574458,1,paintings-orient_107_thmb.jpg
paintings-orient_108_thmb.jpg,150,115,0.275711,0.289569,0.701977,0.095796,0.610242,1,paintings-orient_108_thmb.jpg
paintings-orient_109_thmb.jpg,134,182,0.183852,0.252422,0.685409,0.035527,0.478081,1,paintings-orient_109_thmb.jpg
paintings-orient_111_thmb.jpg,150,120,0.224027,0.345628,0.786914,0.028242,0.544646,1,paintings-orient_111_thmb.jpg
paintings-orient_113_thmb.jpg,150,121,0.170775,0.280903,0.686421,0.023599,0.477444,1,paintings-orient_113_thmb.jpg
...,...,...,...,...,...,...,...,...,...
images-works-89_s.jpg,164,200,0.273511,0.430983,0.650121,0.093226,0.657266,0,images-works-89_s.jpg
images-works-911_s.jpg,151,200,0.160548,0.155590,0.565643,0.055566,0.453683,0,images-works-911_s.jpg
images-works-93_s.jpg,159,200,0.347390,0.208815,0.639162,0.062895,0.380620,0,images-works-93_s.jpg
images-works-95_s.jpg,165,200,0.382387,0.299286,0.636382,0.028307,0.400762,0,images-works-95_s.jpg


In [6]:
dataframe['RMS_contrast'] = dataframe.apply(lambda x: RMS_CONTRAST(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else RMS_CONTRAST(os.path.join(pic_dir,x['Picturesource'])), axis = 1)
dataframe['AVG_entropy'] = dataframe.apply(lambda x: average_entropy(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else average_entropy(os.path.join(pic_dir,x['Picturesource'])), axis = 1)

..\data\henri-matisse\paintings-orient_107_thmb.jpg
..\data\henri-matisse\paintings-orient_108_thmb.jpg
..\data\henri-matisse\paintings-orient_109_thmb.jpg
..\data\henri-matisse\paintings-orient_111_thmb.jpg
..\data\henri-matisse\paintings-orient_113_thmb.jpg
..\data\henri-matisse\paintings-orient_118_thmb.jpg
..\data\henri-matisse\paintings-orient_123_thmb.jpg
..\data\henri-matisse\paintings-orient_127_thmb.jpg
..\data\henri-matisse\paintings-orient_129_thmb.jpg
..\data\henri-matisse\paintings-orient_139_thmb.jpg
..\data\henri-matisse\paintings-orient_141_thmb.jpg
..\data\henri-matisse\paintings-orient_143_thmb.jpg
..\data\henri-matisse\paintings-orient_145_thmb.jpg
..\data\henri-matisse\paintings-orient_150_thmb.jpg
..\data\henri-matisse\paintings-orient_159_thmb.jpg
..\data\henri-matisse\paintings-orient_177_thmb.jpg
..\data\henri-matisse\paintings-orient_179_thmb.jpg
..\data\henri-matisse\paintings-orient_183_thmb.jpg
..\data\henri-matisse\paintings-orient_185_thmb.jpg
..\data\henr

..\data\picasso\images-works-122_s.jpg
..\data\picasso\images-works-1233_s.jpg
..\data\picasso\images-works-125_s.jpg
..\data\picasso\images-works-1263_s.jpg
..\data\picasso\images-works-127_s.jpg
..\data\picasso\images-works-128_s.jpg
..\data\picasso\images-works-137_s.jpg
..\data\picasso\images-works-1450_s.jpg
..\data\picasso\images-works-1453_s.jpg
..\data\picasso\images-works-1459_s.jpg
..\data\picasso\images-works-145_s.jpg
..\data\picasso\images-works-1462_s.jpg
..\data\picasso\images-works-1471_s.jpg
..\data\picasso\images-works-1480_s.jpg
..\data\picasso\images-works-1482_s.jpg
..\data\picasso\images-works-1483_s.jpg
..\data\picasso\images-works-1484_s.jpg
..\data\picasso\images-works-1486_s.jpg
..\data\picasso\images-works-148_s.jpg
..\data\picasso\images-works-1495_s.jpg
..\data\picasso\images-works-14_s.jpg
..\data\picasso\images-works-1513_s.jpg
..\data\picasso\images-works-1518_s.jpg
..\data\picasso\images-works-151_s.jpg
..\data\picasso\images-works-1525_s.jpg
..\data\pi

..\data\picasso\images-works-55_s.jpg
..\data\picasso\images-works-56_s.jpg
..\data\picasso\images-works-57_s.jpg
..\data\picasso\images-works-61_s.jpg
..\data\picasso\images-works-62_s.jpg
..\data\picasso\images-works-64_s.jpg
..\data\picasso\images-works-6_s.jpg
..\data\picasso\images-works-73_s.jpg
..\data\picasso\images-works-77_s.jpg
..\data\picasso\images-works-80_s.jpg
..\data\picasso\images-works-840_s.jpg
..\data\picasso\images-works-85_s.jpg
..\data\picasso\images-works-88_s.jpg
..\data\picasso\images-works-89_s.jpg
..\data\picasso\images-works-911_s.jpg
..\data\picasso\images-works-93_s.jpg
..\data\picasso\images-works-95_s.jpg
..\data\picasso\images-works-97_s.jpg
..\data\henri-matisse\paintings-orient_107_thmb.jpg
..\data\henri-matisse\paintings-orient_108_thmb.jpg
..\data\henri-matisse\paintings-orient_109_thmb.jpg
..\data\henri-matisse\paintings-orient_111_thmb.jpg
..\data\henri-matisse\paintings-orient_113_thmb.jpg
..\data\henri-matisse\paintings-orient_118_thmb.jpg
..\

C:\Users\19099\anaconda3\lib\site-packages\skimage\filters\rank\generic.py:119: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  out_dtype)


..\data\henri-matisse\paintings-orient_139_thmb.jpg
..\data\henri-matisse\paintings-orient_141_thmb.jpg
..\data\henri-matisse\paintings-orient_143_thmb.jpg
..\data\henri-matisse\paintings-orient_145_thmb.jpg
..\data\henri-matisse\paintings-orient_150_thmb.jpg
..\data\henri-matisse\paintings-orient_159_thmb.jpg
..\data\henri-matisse\paintings-orient_177_thmb.jpg
..\data\henri-matisse\paintings-orient_179_thmb.jpg
..\data\henri-matisse\paintings-orient_183_thmb.jpg
..\data\henri-matisse\paintings-orient_185_thmb.jpg
..\data\henri-matisse\paintings-orient_186_thmb.jpg
..\data\henri-matisse\paintings-orient_187_thmb.jpg
..\data\henri-matisse\paintings-orient_189_thmb.jpg
..\data\henri-matisse\paintings-orient_191_thmb.jpg
..\data\henri-matisse\paintings-orient_193_thmb.jpg
..\data\henri-matisse\paintings-orient_195_thmb.jpg
..\data\henri-matisse\paintings-orient_197_thmb.jpg
..\data\henri-matisse\paintings-orient_199_thmb.jpg
..\data\henri-matisse\paintings-orient_211_thmb.jpg
..\data\henr

..\data\picasso\images-works-1480_s.jpg
..\data\picasso\images-works-1482_s.jpg
..\data\picasso\images-works-1483_s.jpg
..\data\picasso\images-works-1484_s.jpg
..\data\picasso\images-works-1486_s.jpg
..\data\picasso\images-works-148_s.jpg
..\data\picasso\images-works-1495_s.jpg
..\data\picasso\images-works-14_s.jpg
..\data\picasso\images-works-1513_s.jpg
..\data\picasso\images-works-1518_s.jpg
..\data\picasso\images-works-151_s.jpg
..\data\picasso\images-works-1525_s.jpg
..\data\picasso\images-works-1526_s.jpg
..\data\picasso\images-works-1528_s.jpg
..\data\picasso\images-works-1529_s.jpg
..\data\picasso\images-works-1535_s.jpg
..\data\picasso\images-works-1543_s.jpg
..\data\picasso\images-works-1550_s.jpg
..\data\picasso\images-works-1562_s.jpg
..\data\picasso\images-works-1563_s.jpg
..\data\picasso\images-works-1575_s.jpg
..\data\picasso\images-works-158_s.jpg
..\data\picasso\images-works-1598_s.jpg
..\data\picasso\images-works-15_s.jpg
..\data\picasso\images-works-1609_s.jpg
..\data

..\data\picasso\images-works-911_s.jpg
..\data\picasso\images-works-93_s.jpg
..\data\picasso\images-works-95_s.jpg
..\data\picasso\images-works-97_s.jpg


In [7]:
df = dataframe
df

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource,RMS_contrast,AVG_entropy
paintings-orient_107_thmb.jpg,150,119,0.245508,0.419425,0.699409,0.057506,0.574458,1,paintings-orient_107_thmb.jpg,0.193308,5.052313
paintings-orient_108_thmb.jpg,150,115,0.275711,0.289569,0.701977,0.095796,0.610242,1,paintings-orient_108_thmb.jpg,0.275974,5.133200
paintings-orient_109_thmb.jpg,134,182,0.183852,0.252422,0.685409,0.035527,0.478081,1,paintings-orient_109_thmb.jpg,0.161583,4.809124
paintings-orient_111_thmb.jpg,150,120,0.224027,0.345628,0.786914,0.028242,0.544646,1,paintings-orient_111_thmb.jpg,0.174502,5.026028
paintings-orient_113_thmb.jpg,150,121,0.170775,0.280903,0.686421,0.023599,0.477444,1,paintings-orient_113_thmb.jpg,0.119690,5.111975
...,...,...,...,...,...,...,...,...,...,...,...
images-works-89_s.jpg,164,200,0.273511,0.430983,0.650121,0.093226,0.657266,0,images-works-89_s.jpg,0.325828,4.745273
images-works-911_s.jpg,151,200,0.160548,0.155590,0.565643,0.055566,0.453683,0,images-works-911_s.jpg,0.229563,4.857225
images-works-93_s.jpg,159,200,0.347390,0.208815,0.639162,0.062895,0.380620,0,images-works-93_s.jpg,0.263236,4.243078
images-works-95_s.jpg,165,200,0.382387,0.299286,0.636382,0.028307,0.400762,0,images-works-95_s.jpg,0.177656,4.960405


In [8]:
dataframe.to_csv (r'../data/classifier.csv', index = False, header=True)

In [9]:
ents = ["mean_hue","mean_saturation","mean_brightness","variance_brightness","sum_edge","RMS_contrast","AVG_entropy"]
X = df[ents]
Y = df["matis"]

In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [13]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# SVC

In [14]:
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
y_pred = clf.predict(x_train)
accuracy_score(y_train, y_pred)

0.5852090032154341

In [16]:
y_pred_test = clf.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.6794871794871795

# Decision Tree Classifier

In [17]:
clf1 = DecisionTreeClassifier(random_state=0)
clf1.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [18]:
y_pred1 = clf1.predict(x_train)
accuracy_score(y_train, y_pred1)

1.0

In [19]:
y_pred1 = clf1.predict(x_train)

In [20]:
y_pred_test = clf1.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.7435897435897436

# Bernoulli NB

In [21]:
clf2 = BernoulliNB()
clf2.fit(x_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [22]:
y_pred2 = clf2.predict(x_train)
accuracy_score(y_train, y_pred2)

0.5884244372990354

In [23]:
y_pred_test = clf2.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.6666666666666666

# K-Neighbors Classifier

In [24]:
from sklearn.neighbors import KNeighborsClassifier
clf3 = KNeighborsClassifier(n_neighbors=4, p=2, leaf_size=60)
clf3.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=60, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [25]:
y_pred3 = clf3.predict(x_train)
accuracy_score(y_train, y_pred3)

0.8263665594855305

In [26]:
y_pred_test = clf3.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.7948717948717948

# Random Forest Classifier

In [27]:
clf4 = RandomForestClassifier(max_depth=14, random_state=10, n_estimators=20)
clf4.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=14, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [28]:
y_pred4 = clf4.predict(x_train)
accuracy_score(y_train, y_pred4)

0.9935691318327974

In [29]:
y_pred_test = clf4.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.8333333333333334